In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from sklearn.datasets import load_iris

In [5]:
load_iris()['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [134]:
X = load_iris()['data']
y = load_iris()['target']
species = np.where(y == 0, 'setosa', np.where(y == 1, 'versicolor', 'virginica'))
data = pd.DataFrame(X, columns = load_iris()['feature_names'])
data['species'] = species

In [135]:
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [136]:
x = data.columns[:-1]
y = data.columns[-1]

In [137]:
def entropy(x):
    px = np.unique(x,return_counts=True)[1]/len(x)
    entropy_ = -np.sum(px * np.log2(px))
    return entropy_

In [138]:
def info_gain(data,x,y):
    total = entropy(data[y])
    gain = []
    for val in np.unique(data[x]):
        info = data.loc[data[x] == val,y]
        count_info = len(info)
        p_info = count_info/len(data)
        gain.append(p_info*entropy(info))
    return total - np.sum(gain)

In [139]:
gains = []
for cols in x:
    gains.append(info_gain(data,cols,'species'))
x[np.argmax(gains)]
    

'petal length (cm)'

In [140]:
def max_gain(data,x,y):
    return x[np.argmax([info_gain(data,i,y) for i in x])]

In [141]:
max_gain(data, x, y)

'petal length (cm)'

In [142]:
x = data.columns[:-1]
y = data.columns[-1]
best_feature = max_gain(data, x, y)

In [143]:
best_feature

'petal length (cm)'

In [176]:
def make_tree(data,x,y,parent_class=None):
    if np.unique(data[y]).size == 1:
        return np.unique(data[y])[0]
    elif len(data) == 0:
        return 'case1'
    elif len(x) == 0:
        return parent_class
    best_feature = max_gain(data,x,y)
    columns = x[x != best_feature]
    tree = {best_feature:{}}
    for val in np.unique(data[best_feature]):
        vals, counts = np.unique(data.loc[data[best_feature] == val,y],return_counts=True)
        parent_class = vals[np.argmax(counts)]
        sub_data = data.loc[data[best_feature] == val,:]
        sub_tree = make_tree(sub_data,columns,y,parent_class)
        tree[best_feature][val] = sub_tree
    return tree

In [145]:
data[best_feature][np.argmin(data[best_feature])]
data[best_feature][np.argmax(data[best_feature])]

6.9

In [146]:
petal_list = np.arange(data[best_feature][np.argmin(data[best_feature])], data[best_feature][np.argmax(data[best_feature])], 0.5).tolist()

In [147]:
minimum = data[best_feature][np.argmin(data[best_feature])]
maximum = data[best_feature][np.argmax(data[best_feature])]
best_feature_list = np.arange(minimum, maximum, 0.5)
for val in best_feature_list:
    print(val)

1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
6.0
6.5


In [163]:
best_feature

'petal length (cm)'

In [173]:
data[best_feature] < 1+0.5

0       True
1       True
2       True
3      False
4       True
       ...  
145    False
146    False
147    False
148    False
149    False
Name: petal length (cm), Length: 150, dtype: bool

In [162]:
data.loc[1 <= data[best_feature] < 1+0.5,y]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [177]:
make_tree(data,x,y)

{'petal length (cm)': {1.0: 'setosa',
  1.1: 'setosa',
  1.2: 'setosa',
  1.3: 'setosa',
  1.4: 'setosa',
  1.5: 'setosa',
  1.6: 'setosa',
  1.7: 'setosa',
  1.9: 'setosa',
  3.0: 'versicolor',
  3.3: 'versicolor',
  3.5: 'versicolor',
  3.6: 'versicolor',
  3.7: 'versicolor',
  3.8: 'versicolor',
  3.9: 'versicolor',
  4.0: 'versicolor',
  4.1: 'versicolor',
  4.2: 'versicolor',
  4.3: 'versicolor',
  4.4: 'versicolor',
  4.5: {'sepal length (cm)': {4.9: 'virginica',
    5.4: 'versicolor',
    5.6: 'versicolor',
    5.7: 'versicolor',
    6.0: 'versicolor',
    6.2: 'versicolor',
    6.4: 'versicolor'}},
  4.6: 'versicolor',
  4.7: 'versicolor',
  4.8: {'sepal length (cm)': {5.9: 'versicolor',
    6.0: 'virginica',
    6.2: 'virginica',
    6.8: 'versicolor'}},
  4.9: {'sepal width (cm)': {2.5: 'versicolor',
    2.7: 'virginica',
    2.8: 'virginica',
    3.0: 'virginica',
    3.1: 'versicolor'}},
  5.0: {'sepal length (cm)': {5.7: 'virginica',
    6.0: 'virginica',
    6.3: 'virgini